### Importaciones de paquetes necesarios

In [134]:
from deap import creator, base, tools, algorithms

import random
import numpy as np
import pandas as pd
import itertools

### Creación del turno de enfermería

In [32]:
from turnosenfermeria import TurnosEnfermeria

In [33]:
preferencias = pd.read_csv("./data/datos_enfermeria.txt", sep=' ')
print (preferencias)

   Nombre  Pref_M  Pref_T  Pref_N
0   Enf_A       1       1       0
1   Enf_B       1       0       1
2   Enf_C       1       1       1
3   Enf_D       0       1       0
4   Enf_E       0       0       1
5   Enf_F       0       1       1
6   Enf_G       1       1       0
7   Enf_H       0       0       1
8   Enf_I       1       1       1
9   Enf_J       1       1       0
10  Enf_K       1       1       1
11  Enf_L       1       1       0
12  Enf_M       1       0       1


In [34]:
enfermeras = preferencias["Nombre"]
print (enfermeras)

0     Enf_A
1     Enf_B
2     Enf_C
3     Enf_D
4     Enf_E
5     Enf_F
6     Enf_G
7     Enf_H
8     Enf_I
9     Enf_J
10    Enf_K
11    Enf_L
12    Enf_M
Name: Nombre, dtype: object


In [60]:
turnos = TurnosEnfermeria (enfermeras, preferencias)




273

# Algoritmo Genético Simple

In [247]:
tamano_individuo = 21
tamano_calendario = len (preferencias)
tamano_poblacion = 50
max_generaciones = 20
p_cruce = 0.9
p_mutacion = 0.1

toolbox = base.Toolbox()

In [248]:
# Establecemos cual va a ser la función de ajuste e inicializamos los individuos y la población

creator.create("ClaseAjusteMin", base.Fitness, weights=(-1.0,))
creator.create("ClaseIndividuo", list, fitness=creator.ClaseAjusteMin)

toolbox.register("ceroOrUno", random.randint, 0, 1)
toolbox.register("individual", tools.initRepeat, creator.ClaseIndividuo, toolbox.ceroOrUno, tamano_individuo)
toolbox.register("population", tools.initRepeat, list, toolbox.individual, tamano_poblacion)
poblacion = toolbox.population()

toolbox.register("evaluate", turnos.getCoste)

In [249]:
# Establecemos los operadores de selección, cruce y mutación

toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("mate", tools.cxOnePoint)
toolbox.register("mutate", tools.mutFlipBit, indpb=1.0/tamano_individuo)

In [250]:
#Incluimos el objeto Statistics

stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("avg", np.mean)
stats.register("std", np.std)
stats.register("max", np.max)
stats.register("min", np.min)

In [251]:
#Incluimos el objeto Hall Of Fame para saber cual es el mejor individuo en cada generación

hof = tools.HallOfFame(1)


## Vamos a crear el Algoritmo Genético Simple a ver que nos devuelve

In [252]:
def getCosteIndividual (self, individuo):
    turnosEnfermDict = self.getTurnosEnfermeria(individuo)

        
    infracionesTurnosConsecutivos = InfracionesTurnosConsecutivos(turnosEnfermDict)
    infracionesTurnosMismoDia = InfracionesTurnosMismoDia(turnosEnfermDict)
    infraccionesTurnosPorSemana = InfraccionesTurnosPorSemana(turnosEnfermDict)[1]
    infraccionesEnfermerPorTurno = InfraccionesEnfermerPorTurno(turnosEnfermDict)[1]
    infraccionesPreferenciaTurnos = InfraccionesPreferenciaTurnos(turnosEnfermDict)[1]

        
    infraccionesRestriccionesHard = infracionesTurnosConsecutivos + infracionesTurnosMismoDia + infraccionesEnfermerPorTurno + infraccionesTurnosPorSemana
    infraccionesRestriccionesSoft = infraccionesPreferenciaTurnos

    return turnos.factorPenalizacionRestriccionDura * infraccionesRestriccionesHard + infraccionesRestriccionesSoft,
    

In [240]:



        turnosEnfermDict = self.getTurnosEnfermeria(calendario)

        
        infracionesTurnosConsecutivos = self.countInfracionesTurnosConsecutivos(turnosEnfermDict)
        infracionesTurnosMismoDia = self.countInfracionesTurnosMismoDia(turnosEnfermDict)
        infraccionesTurnosPorSemana = self.countInfraccionesTurnosPorSemana(turnosEnfermDict)[1]
        infraccionesEnfermerPorTurno = self.countInfraccionesEnfermerPorTurno(turnosEnfermDict)[1]
        infraccionesPreferenciaTurnos = self.countInfraccionesPreferenciaTurnos(turnosEnfermDict)[1]

        
        infraccionesRestriccionesHard = infracionesTurnosConsecutivos + infracionesTurnosMismoDia + infraccionesEnfermerPorTurno + infraccionesTurnosPorSemana
        infraccionesRestriccionesSoft = infraccionesPreferenciaTurnos

        return self.factorPenalizacionRestriccionDura * infraccionesRestriccionesHard + infraccionesRestriccionesSoft,

In [246]:
def CreaccionCalendarioTurnos (tamano_individuo, tamano_poblacion, p_cruce, p_mutacion, max_generaciones, verbose=True):
    toolbox=base.Toolbox()
    creator.create("ClaseAjusteMin", base.Fitness, weights=(-1.0,))
    creator.create("ClaseIndividuo", list, fitness=creator.ClaseAjusteMin)

    toolbox.register("ceroOrUno", random.randint, 0, 1)
    toolbox.register("individual", tools.initRepeat, creator.ClaseIndividuo, toolbox.ceroOrUno, tamano_individuo)
    toolbox.register("population", tools.initRepeat, list, toolbox.individual, tamano_poblacion)
    poblacion = toolbox.population()
   


    toolbox.register("evaluate", turnos.getCoste)


    toolbox.register("select", tools.selTournament, tournsize=3)
    toolbox.register("mate", tools.cxOnePoint)
    toolbox.register("mutate", tools.mutFlipBit, indpb=1.0/tamano_individuo)


    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", np.mean)
    stats.register("std", np.std)
    stats.register("max", np.max)
    stats.register("min", np.min)


    hof = tools.HallOfFame(1)
    poblacion_final, logbook= algorithms.eaSimple(poblacion, toolbox,
                                                 cxpb=p_cruce, mutpb=p_mutacion,
                                                 ngen=max_generaciones,
                                                 stats=stats,halloffame=hof,
                                                 verbose=verbose)

    return poblacion_final, logbook

In [245]:
# Comprobación del funcionamiento del algoritmo


poblacion_final, logbook = CreaccionCalendarioTurnos (tamano_individuo, tamano_poblacion, p_cruce, p_mutacion, max_generaciones)

[[0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0], [1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 

TypeError: Both weights and assigned values must be a sequence of numbers when assigning to values of <class 'deap.creator.ClaseAjusteMin'>. Currently assigning value(s) ((189.28000000000003,),) of <class 'tuple'> to a fitness with weights (-1.0,).

In [243]:
print(len(turnos))

273
